In [38]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import scipy as sp
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.svm import SVC

In [39]:
def read_data(folder, name1, name2, name3):
    loc = "%s/%s.txt" %(folder, name1)
    train_set = pd.read_csv(loc, sep="\t")
    print "Training set has %s rows and %s coumns" %(train_set.shape[0], train_set.shape[1])
    loc = "%s/%s.txt" %(folder, name2)
    test_set = pd.read_csv(loc, sep="\t")
    print "Testing set has %s rows and %s coumns" %(test_set.shape[0], test_set.shape[1])
    loc = "%s/%s.txt" %(folder, name3)
    val_set = pd.read_csv(loc, sep="\t")
    print "Validation set has %s rows and %s coumns" %(val_set.shape[0], val_set.shape[1])
    return train_set, test_set, val_set

In [40]:
train_set, test_set, val_set = read_data("data3", "train_data", "test_data", "valid_data")

Training set has 762 rows and 25 coumns
Testing set has 254 rows and 25 coumns
Validation set has 253 rows and 25 coumns


In [41]:
train_set_nb = sp.genfromtxt("data3/train_data.txt", delimiter="\t")
test_set_nb = test_set.values
val_set_nb = val_set.values

In [42]:
def np_cleaner(train, test, val):
    train = train[~np.isnan(train).any(axis=1)]
    print "Training set has %s rows and %s coumns" %(train.shape[0], train.shape[1])
    test = test[~np.isnan(test).any(axis=1)]
    print "Testing set has %s rows and %s coumns" %(test.shape[0], test.shape[1])
    val = val[~np.isnan(val).any(axis=1)]
    print "Validation set has %s rows and %s coumns" %(val.shape[0], val.shape[1])
    return train, test, val

In [44]:
train_data = train_set_nb[~np.isnan(train_set_nb).any(axis=1)]

In [52]:
train_set_nb, test_set_nb, val_set_nb = np_cleaner(train_set_np, test_set_np, val_set_np)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [45]:
train_set.columns

Index([u'GP', u'GS', u'MIN', u'FGM', u'FGA', u'FG%', u'3PM', u'3PA', u'3P%',
       u'FTM', u'FTA', u'FT%', u'OFF', u'DEF', u'TRB', u'AST', u'STL', u'BLK',
       u'PF', u'TOV', u'PTS', u'YR', u'POS', u'W', u'H'],
      dtype='object')

In [46]:
columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"POS":22,"W":23,"H":24} 

# Labels

In [47]:
def np_labeliser(train, test, val, col):
    train = train[:,col]
    test = test[:,col]
    val = val[:,col]
    return train, test, val

In [48]:
train_labels, test_labels, val_labels = np_labeliser(train_set_nb, test_set_nb, val_set_nb, 22)

# Feature Selection

In [49]:
def np_featuriser(train, test, val,feature_lsit):
    
    train = np.delete(train,feature_list,1)
    test = np.delete(test,feature_list,1)
    val = np.delete(val,feature_list,1)
    
    return train, test, val

In [50]:
feature_list = [22]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

In [51]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=10)
X = train_features_nb
y = train_labels
#print X[:1]
selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
#print scores
scores /= scores.max()
print scores

#for i in scores:
#scores

print "Most valuable features: "
remove = []
for i in range(0,25):
    remove.append(i)
    
j = 0
for i in scores:
    
    if i>0.08:
        print j, i, columns.keys()[columns.values().index(j)]
        remove.remove(j)
    if j == 21:
        j = j+2
    else:
        j = j+1
#for i in range(0,25)
print remove

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').